In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs
from dateutil import parser

In [2]:
data = {
    "keywords": "anies",
    "since_time": "2023-09-01",
    "until_time": "2023-11-10"
}

In [3]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
url = f"https://index.sindonews.com/index/0?t=2023-09-01"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
# print(soup)
cek = soup.find('div', {"class": "pagination"})
links=cek.find_all('a')
all_link=[]
for link in links:
        if link and 'href' in link.attrs:
            link_href = link['href']
            all_link.append(link_href)
link=all_link[-1]
# print(link)
parsed_url = urlparse(link)
number = parsed_url.path.split("/")[-1]
print(number)

440


In [4]:
def scrape_links(date, page_number):
    input_date_str = datetime.strftime(date, "%Y-%m-%d")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://index.sindonews.com/index/5/{page_number}?t={input_date_str}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', {"class": "indeks-title"})
    
    links = []
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
        
    print(f"Scraped {len(links)} links from page {page_number} url {url}")
    return links

In [5]:

def scrape_link_per_day(date, max_threads=5):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    current_date = datetime.strptime(date, "%Y-%m-%d")
    url = f"https://index.sindonews.com/index/5/0?t={current_date}"
    response = requests.get(url + "1", headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    cek = soup.find('div', {"class": "pagination"})
    links = cek.find_all('a')
    all_link = [link['href'] for link in links if link and 'href' in link.attrs]

    if not all_link:
        return []

    last_link = all_link[-1]

    # Extract the "page" parameter value from the last link
    parsed_url = urlparse(last_link)
    page_value = parsed_url.path.split("/")[-1]
    print(page_value)
    # Check if page_value is an integer
    try:
        page_number = int(page_value)
    except (TypeError, ValueError):
        page_number = 0

    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        # Use list comprehension to submit tasks to the thread pool
        futures = [executor.submit(scrape_links, current_date, index) for index in range(0, page_number + 20, 20)]

        # Collect results from completed futures
        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [6]:
link=scrape_link_per_day("2023-09-01")

180
Scraped 20 links from page 20 url https://index.sindonews.com/index/5/20?t=2023-09-01
Scraped 20 links from page 40 url https://index.sindonews.com/index/5/40?t=2023-09-01
Scraped 2 links from page 80 url https://index.sindonews.com/index/5/80?t=2023-09-01
Scraped 20 links from page 60 url https://index.sindonews.com/index/5/60?t=2023-09-01
Scraped 20 links from page 0 url https://index.sindonews.com/index/5/0?t=2023-09-01
Scraped 0 links from page 120 url https://index.sindonews.com/index/5/120?t=2023-09-01
Scraped 0 links from page 100 url https://index.sindonews.com/index/5/100?t=2023-09-01
Scraped 0 links from page 140 url https://index.sindonews.com/index/5/140?t=2023-09-01
Scraped 0 links from page 160 url https://index.sindonews.com/index/5/160?t=2023-09-01
Scraped 0 links from page 180 url https://index.sindonews.com/index/5/180?t=2023-09-01


In [7]:
print(len(link))

82


In [8]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('div', {"class": "detail-title"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"   
                    # tanggal berita
                    date_elem = soup.find('div', {"class": "detail-date-artikel"})
                    if date_elem:
                        date_text = date_elem.text.strip()
                        date_part = ' '.join(date_text.split(',')[1:]).strip()
                        date_object = parser.parse(date_part)
                        formatted_date = date_object.strftime('%Y-%m-%d')
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"class": "detail-desc"})
                    
                    if body_elem:
                        content_text = body_elem.text
                        content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                        content_text = ' '.join(content_text.split())
                    else:
                        content_text ="Content not found"

                    return{
                        'title': title_text,
                        'date': formatted_date,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [9]:
cek = scrape_url('https://nasional.sindonews.com/read/1257753/14/lanjutkan-program-yudo-margono-panglima-tni-akan-naikkan-tunjangan-lauk-pauk-prajurit-1700629885')

In [10]:
print(cek)

{'title': 'Lanjutkan Program Yudo Margono, Panglima TNI Akan Naikkan Tunjangan Lauk-pauk Prajurit', 'date': '2023-11-22', 'content': 'JAKARTA - Panglima TNI Jenderal TNI Agus Subiyanto akan melanjutkan program-program Panglima TNI sebelumnya Laksamana TNI Yudo Margono . Ia berjanji akan menaikkan tunjangan lauk-pauk prajurit TNI."Mungkin saya juga akan melanjutkan program-program Panglima TNI yang lalu (Laksamana Yudo Margono) dan sesuai visi-misi waktu fit and proper test. Jadi ada visi dan misi saya adalah TNI yang prima yaitu profesional, responsif, integratif, modern dan adaptif," ujar Agus dalam keterangannya usai pelantikan di Istana Negara, Jakarta, Rabu (22/11/2023). Baca JugaSiapa Calon KSAD? Panglima TNI Agus Subiyanto: Kita Lihat Saja Bintang 3 Eligible Agus juga akan menaikkan tunjangan lauk-pauk prajurit TNI. Hal tersebut telah disampaikannya kepada Kementerian Pertahanan (Kemhan)."Jadi tunjangannya harus dinaikkan. Nanti secara button up saya akan mengajukan kepada Kemhan

In [11]:
# for url in link:
#     data_tribunnews = scrape_url(url)
#     print(data["keywords"])
#     keywords=data["keywords"]
#     if keywords.lower() in data_tribunnews['title'].lower() or keywords.lower() in data_tribunnews['content'].lower():
#         print("Data contains keywords:", data_tribunnews['link'])
#     else:
#         print("News does not contain the specified keywords and will not be inserted into the database. URL:", data_tribunnews['link'])